In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

file_path = '/Users/anishanburaja/Downloads/amazon 3.csv'
df = pd.read_csv(file_path)

features1 = ['discounted_price']  
features2 = ['rating']  

for col in features1 + features2:
    df[col] = pd.to_numeric(df[col].replace('[₹,]', '', regex=True), errors='coerce')
df = df.dropna(subset=features1 + features2).reset_index(drop=True)

if df.empty:
    print("DataFrame is empty after cleaning.")
else:
    target_product_index = min(0, len(df) - 1)  
    target_product = df.loc[target_product_index, features1 + features2].astype(float)
    product_details = df[['product_name', 'category']].values.tolist()

    price_distances = euclidean_distances(df[features1], [target_product[0:1]])[:, 0]
    rating_distances = euclidean_distances(df[features2], [target_product[1:2]])[:, 0]

    def calculate_jaccard_similarity(category_A, category_B):
        set_A = set(category_A.split(','))
        set_B = set(category_B.split(','))
        intersection = len(set_A.intersection(set_B))
        union = len(set_A.union(set_B))
        return intersection / union if union != 0 else 0

    jaccard_similarities = [
        calculate_jaccard_similarity(df.loc[target_product_index, 'category'], df.loc[idx, 'category']) 
        for idx in range(len(df))
    ]

    price_similar_df = pd.DataFrame(
        [(product_details[idx][0], product_details[idx][1], distance) for idx, distance in sorted(zip(df.index, price_distances), key=lambda x: x[1])[:10]],
        columns=['Product Name', 'Category', 'Price Diff']
    )

    rating_similar_df = pd.DataFrame(
        [(product_details[idx][0], product_details[idx][1], distance) for idx, distance in sorted(zip(df.index, rating_distances), key=lambda x: x[1])[:10]],
        columns=['Product Name', 'Category', 'Rating Diff']
    )

    category_similar_df = pd.DataFrame(
        [(product_details[idx][0], product_details[idx][1], similarity) for idx, similarity in sorted(enumerate(jaccard_similarities), key=lambda x: x[1], reverse=True)[:10]],
        columns=['Product Name', 'Category', 'Jaccard Similarity']
    )

    print("Most Similar by Price:")
    display(price_similar_df)

    print("\nMost Similar by Rating:")
    display(rating_similar_df)

    print("\nMost Similar by Jaccard Similarity:")
    display(category_similar_df)

    combined_similarity_score = 0.4 * price_distances + 0.3 * rating_distances + 0.3 * (1 - np.array(jaccard_similarities))
    combined_similarity_indices = np.argsort(combined_similarity_score)[:10]

    top_10_similar_df = pd.DataFrame(
        [(product_details[idx][0], product_details[idx][1], combined_similarity_score[idx]) for idx in combined_similarity_indices],
        columns=['Product Name', 'Category', 'Combined Similarity Score']
    )

    print("Most Similar by Combined Score:")
    display(top_10_similar_df)


Most Similar by Price:


,Product Name,Category,Price Diff
0,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,0.0
1,Wayona Nylon Braided 3A Lightning to USB A Syn...,Computers&Accessories|Accessories&Peripherals|...,0.0
2,7SEVEN® Compatible for Samsung Smart 4K Ultra ...,"Electronics|HomeTheater,TV&Video|Accessories|R...",0.0
3,"boAt Type C A750 Stress Resistant, Tangle-free...",Computers&Accessories|Accessories&Peripherals|...,0.0
4,Wayona Usb Nylon Braided Data Sync And Chargin...,Computers&Accessories|Accessories&Peripherals|...,0.0
5,Wayona Type C To Type C Long Fast Charging Cab...,Computers&Accessories|Accessories&Peripherals|...,0.0
6,"boAt Type C A750 Stress Resistant, Tangle-free...",Computers&Accessories|Accessories&Peripherals|...,0.0
7,Cotbolt Silicone Protective Case Cover for LG ...,"Electronics|HomeTheater,TV&Video|Accessories|R...",0.0
8,Wayona Usb Type C 65W 6Ft/2M Long Fast Chargin...,Computers&Accessories|Accessories&Peripherals|...,0.0
9,LRIPL Compatible Sony Bravia LCD/led Remote Wo...,"Electronics|HomeTheater,TV&Video|Accessories|R...",0.0



Most Similar by Rating:


,Product Name,Category,Rating Diff
0,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,0.0
1,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,0.0
2,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,0.0
3,"TP-Link USB WiFi Adapter for PC(TL-WN725N), N1...",Computers&Accessories|NetworkingDevices|Networ...,0.0
4,boAt Rugged v3 Extra Tough Unbreakable Braided...,Computers&Accessories|Accessories&Peripherals|...,0.0
5,Portronics Konnect CL 20W POR-1067 Type-C to 8...,Computers&Accessories|Accessories&Peripherals|...,0.0
6,MI 80 cm (32 inches) 5A Series HD Ready Smart ...,"Electronics|HomeTheater,TV&Video|Televisions|S...",0.0
7,Tizum High Speed HDMI Cable with Ethernet | Su...,"Electronics|HomeTheater,TV&Video|Accessories|C...",0.0
8,OnePlus 80 cm (32 inches) Y Series HD Ready LE...,"Electronics|HomeTheater,TV&Video|Televisions|S...",0.0
9,OnePlus 126 cm (50 inches) Y Series 4K Ultra H...,"Electronics|HomeTheater,TV&Video|Televisions|S...",0.0



Most Similar by Jaccard Similarity:


,Product Name,Category,Jaccard Similarity
0,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,1.0
1,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,1.0
2,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,1.0
3,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,1.0
4,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,1.0
5,pTron Solero TB301 3A Type-C Data and Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,1.0
6,"boAt Micro USB 55 Tangle-free, Sturdy Micro US...",Computers&Accessories|Accessories&Peripherals|...,1.0
7,MI Usb Type-C Cable Smartphone (Black),Computers&Accessories|Accessories&Peripherals|...,1.0
8,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,1.0
9,Portronics Konnect L POR-1081 Fast Charging 3A...,Computers&Accessories|Accessories&Peripherals|...,1.0


Most Similar by Combined Score:


,Product Name,Category,Combined Similarity Score
0,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,0.00
1,Wayona Nylon Braided Lightning USB Data Sync &...,Computers&Accessories|Accessories&Peripherals|...,0.00
2,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,0.00
3,Wayona Nylon Braided 3A Lightning to USB A Syn...,Computers&Accessories|Accessories&Peripherals|...,0.00
4,Wayona Usb Nylon Braided Data Sync And Chargin...,Computers&Accessories|Accessories&Peripherals|...,0.00
5,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,0.00
6,"boAt Type C A750 Stress Resistant, Tangle-free...",Computers&Accessories|Accessories&Peripherals|...,0.03
7,"boAt Type C A750 Stress Resistant, Tangle-free...",Computers&Accessories|Accessories&Peripherals|...,0.03
8,Wayona Usb Type C 65W 6Ft/2M Long Fast Chargin...,Computers&Accessories|Accessories&Peripherals|...,0.03
9,Zoul Type C to Type C Fast Charging Cable 65W ...,Computers&Accessories|Accessories&Peripherals|...,0.03
